# Earth Data Analytics Bootcamp Final Assignment

---

## Grassland Management Under Climate Change - A Habitat Sustainability Model for Native Grass, Sorghastrum Nutans

![Sorghastrum nutans](https://www.wildflower.org/image_archive/640x480/SAW/SAW_03643.JPG)

Photograph of Sorghastrum nutans, Indiangrass. Photo Credits: [Wasowski Collection](https://www.wildflower.org/gallery/result.php?id_image=24692)

Native grasslands account for nearly a quarter of ground surface vegetative cover, but have suffered significant global losses in habitat. While anthropogenic encroachment has contributed to these losses, dramatic changes in climate over the last few decades has taken a toll on grassland ecosystems (Bond, 2008). Consequently, native grasslands are endangered ecosystems, posing economic and ecological threats. Unfortunately, given the complications of a changing climate, restoration of native grasslands is all but simple. Efforts to rebuild and conserve grassland ecosystems now require careful planning and must account for future changes in climate inorder to build ecologic resilence (Kane et al., 2017). Ecologic planning will have to account for longer duration droughts, increased temperatures, and more variable extreme precipitation events predicted by climate forecasts. The time available to understand, conserve, and restore these critical ecosystems are shrinking rapidly, required immediate action.

The purpose of this project is to create a habitat suitability model for one variety of native grass, Sorghastrum nutans, more commonly known as Indiangrass, is a native, warm season perennial grass commonly found between the east coast and Rocky Mountains in the United States. It is the second most abundant tall grass species in North America [(Silletti, 2002)](https://www.jstor.org/stable/pdf/20051307.pdf?casa_token=1qAlFkV1krQAAAAA:wBQHle8JfCNPdjWcJwl08FTB5yWfkqHSfeQEHA_U6DwX2KPx7xwEhE0_PoXB3_VT4DllfADhZhIdcUEzsJFcXXx-whL2kFuiWMg1iFaf_E6AnDBSnTIn). 

Indiangrass grows between 3-8 feet tall, has 10-20 inch leaf blades and a hairy stalk. For habitat, Indiangrass often occurs in regions receiving 11-45 inches of annual precipitation, preferrs deep, moist soils ranging from heavy clays to sands, and a pH range of 4.8 - 8.0. However, the species is has a moderate tolerance for salinity, drought, and periodic burning [(Brakie, 2017)](https://www.nrcs.usda.gov/plantmaterials/etpmcpg13196.pdf). Studies of long term abundance patterns and responses of Sorghastrum nutans to climate and environmental changes suggest that variations in annual precipitation impacted ground cover, but that the response to increased fire frequency was an increase in ground cover, indicating species resilience (Silletti, 2002).

**References**

Bond, W.J. (2008). What limits trees in C4 grasslands adn savannas? Annual Review of Ecology, Evolution, and Systematics. v39, p641-659. doi: 10.1146/annurev.ecolsys.39.110707.173411

Brakie, M. 2017. Plant Guide for Indiangrass (Sorghastrum nutans). USDA-Natural Resources Conservation Service, East
Texas Plant Materials Center. Nacogdoches, TX 75964.

Kane, Kristen, Diane Debinski, Chris Anderson, John Scasta, David Engle, and James Miller (2017). Using regional climate projections to guide grassland community restoration in the face of climate change. Frontiers Plant Science. v8. https://doi.org/10.3389/fpls.2017.00730

Silletti, A., & Knapp, A. (2002). Long-term responses of the grassland co-dominants Andropogon gerardii and Sorghastrum nutans to changes in climate and management. Plant Ecology, 163, 15-22.

In [ ]:
# Import standard libraries
import logging              # Logging of messages
import os                   # Handles files and directories
import shutil               # Clear cashe
import warnings             # Show warning triggered by module import
from glob import glob       # Creates lists of files for batch processing

# Import third party libraries
import earthpy as et        # Earthpy library
import earthpy.appeears as eaapp   # NASA AppEEARS Data Download
import earthpy.earthexplorer as etee    # Plot and manipulate spatial data
import earthpy.spatial as es    # Used for spatial statistics
import geopandas as gpd     # Work with vector files
import geoviews as gv       # Visualization package
import holoviews as hv      # Visualization package
import hvplot.pandas        # Used for plotting data
import hvplot.xarray        # Used for plotting maps
import numpy as np          # Used for math operations
import pandas as pd         # Used for working with tabular data
import requests             # Request data to/from server
import rioxarray as rxr     # Storage and manipulation of rasters
import rioxarray.merge as rxrm   # Merge rioxarrays
import xarray as xr         # Work with multidimensional arrays

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning
# warnings.simplefilter(action='ignore')

# Get root directory and data directory paths and save as variables
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
grassland_dir = os.path.join(data_dir, 'grassland-units')
soil_dir = os.path.join(data_dir, 'polaris-soils')
elevation_dir = os.path.join(data_dir, 'srtm-elevation')
climate_dir = os.path.join(data_dir, 'macav2-climate')

# If directories do not exist, create
for a_dir in [grassland_dir, soil_dir, elevation_dir, climate_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

In [ ]:
# Check that data directories were created
print(f'\n Root Directory: {data_dir}', 
      f'\n Grassland Data Directory: {grassland_dir}',
      f'\n Soil Data Directory: {soil_dir}',
      f'\n Elevation Data Directory: {elevation_dir}',
      f'\n Climate Data Directory: {climate_dir}')

## USFS National Grassland Units

---

The [United States Forest Service administers twenty grassland units](https://www.fs.usda.gov/managing-land/national-forests-grasslands/national-grasslands) in twelve states composing over 3.8 million acres of public land. Each unit serves a variety of purposes related to ecology, biology, resources, and recreation. Native grasslands also provide a variety of [ecosystem services](https://www.fs.usda.gov/managing-land/national-forests-grasslands/national-grasslands/ecoservices) that benefit the health and wellness of humans and the natural world. These include but are not limited to: drought and flood mitigation, development of soil nutrients and fertility management, pest control, protection of natural water systems and from soil erosion, and sequestration of atmospheric carbon dioxide (CO2) among others.

This study focuses on two national grassland units near the west coast of the United States: Butte Valley National Grassland in California and Crooked River National Grassland in Oregon. Despite their similar longitudes, differences in latitudes may impact their respective habitat sustainability models.

### [Butte Valley National Grassland - California](https://www.fs.usda.gov/detail/klamath/about-forest/?cid=FSEPRD494406)

Butte Valley National Grassland is located in a scenic basin lowland in the southern Casade Range, northern California. It is California's only national grassland. The grassland is 18.425 acres in size and plays host to sagebrush, rabbitbrush, bitterbrush, basin wildrye, intermediate wheatgrass, a variety of arid grasses and flowers, and occassional western Juniper trees.

![Butte Valley Grassland](https://github.com/ellenalamont17/bootcamp_final_2023-ellenalamont17/IMG/ButteValleyGrassland.jpg)

*Photograph of the Butte Valley National Grassland looking toward Mount Shasta with a view of intermediate wheatgrass in the foreground. Photo Credit: [U.S. Forest Service](http://www.fs.fed.us/grasslands/lonetree/buttevalley.shtml)*


### [Crooked River National Grassland - Oregon](https://www.fs.usda.gov/recarea/ochoco/recarea/?recid=38274)

Crooked River National Grassland is located west of the central Cascade Range in Jefferson County, Oregon. It is Oregon's only national grassland. The grassland is 173,629 acres and contains two National Wild and Scenic Rivers - the Deschutes and Crooked rivers. Crested wheatgrass, bluebunch wheatgrass, and Idaho fescue are common.

![Crooked River Grassland](https://github.com/ellenalamont17/bootcamp_final_2023-ellenalamont17/IMG/CrookedRiverGrassland.jpg)

*Photograph of the Crooked River National Grassland looking from pinnacle ridge toward Three Sisters volcanoes. Photo Credit: [The Oregonian](https://www.oregonlive.com/terryrichard/2011/12/crooked_river_grassland_is_no.html)*

In [ ]:
# Download, open, read, and select grassland units 

# Download the grassland unit shapefile if it does not exist
grassland_path = os.path.join(grassland_dir, 'S_USA.NationalGrassland.shp')

if not os.path.exists(grassland_path):
    grass_zip_url = ('https://data.fs.usda.gov/geodata/edw/edw_resources/'
                     'shp/S_USA.NationalGrassland.zip')
    gpd.read_file(grass_zip_url).to_file(grassland_path)
# grassland_path                    # Test if shapefile was downloaded to path

# Load the grassland bounary data into a gdf and set the index
grassland_bndry = gpd.read_file(grassland_path).set_index('GRASSLANDN')
# grassland_bndry                   # Test if shapefile was read to gdf


# Select specific grassland unit boundary rows
unit_bndry = (
    grassland_bndry
    .loc[['Butte Valley National Grassland',
           'Crooked River National Grassland']]
)
unit_bndry                          # Test if individual units were selected

In [ ]:
# Reproject grassland unit bounaries to CRS 3857 - Web Mercator
unit_bndry_rpj = unit_bndry.to_crs({'init':'epsg:3857'})
# grassland_bndry_rpj = grassland_bndry.to_crs({'init':'epsg:3857'})


# Print the CRS of each file after reprojection
# Tests that reprojection was successful
print('Geodetic CRS of grassland boundary before reprojection: ' 
      + str(unit_bndry.crs))
print('Geodetic CRS of grassland boundary after reprojection: ' 
      + str(unit_bndry_rpj.crs))
  

# Create a map of the select grassland boundaries
boundary_map = unit_bndry_rpj.hvplot(
    x='x', y='y',
    xlabel='Longitude (decimal degrees)',
    ylabel='Latitude (decimal degrees)',
    title='Grassland Unit Boundaries for Habitat Sustainability Model',
    aspect='equal',
    height=600,
    width=600,
    xformatter='%.0f',
    yformatter='%.0f',
    line_color='black',
    line_width=2,
    fill_color='none',
    tiles='OSM')

boundary_map

# hv.save(boundary_map, 'boiseboundarymap.html')

In [ ]:
# Extract the bounds (lat/lon) for individual grassland units 
unit_lat_lon = unit_bndry.bounds
# unit_lat_lon = unit_lat_lon.reset_index()
# unit_lat_lon

# Create empty list to hold grassland unit coordinates
butte_coord = []                    # Coordinates of Individual Unit
crooked_coord = []                  # Coordinates of Individual Unit
coord_list = []                     # Composite coordinates for looping

# Create Lists of Min/Max Lat/Lon
min_lon= (unit_lat_lon[['minx']].values)
max_lon= (unit_lat_lon[['maxx']].values)
min_lat= (unit_lat_lon[['miny']].values)
max_lat= (unit_lat_lon[['maxy']].values)

# Iterrate through the units to extract specific unit coordinate bounds
# Note that indices are min and max longitude and min and max latitude
count=0
for unit_name, unit_row in unit_lat_lon.iterrows():    
      # Extract values from the given index number (count)
      unit_coords = [
            min_lon[count],max_lon[count],min_lat[count],max_lat[count]]
      # Assign index zero values to Butte Valley Grassland
      if count==0:
            butte_coord = unit_coords
            print('Butte Valley Coordinates Calculated')
            # print(butte_coord)
      # Assign index one values to Crooked River Grassland
      elif count==1:
            crooked_coord = unit_coords
            print('Crooked River Coordinates Calculated')
            # print(crooked_coord)
      # If there are additional index values, throw an error
      else:
            print('Too many coordinate sets. Confirm number of units')
      
      coord_list.append(unit_coords)
      count=count+1



## Habitat Sustainability Model

---

**Explain what is a HSM**

### **Model Parameters: Soil, Elevation, and Climate**

### POLARIS Soil Data

The POLARIS soil data is a 1 arc second (30-meter) spatial resolution probabilistic soil series map of the contiguous United States. The seamless, internally consistent soil map was created using published environmental data and machine learning algorithms to remap the Soild Survey Geographic (SSURGO) database. Statistics are provided for each of the data variables listed below and include the mean, mode, median, and the 5th and 95th percentiles.

Mapped variables include: 

| Variable | Description| Units |
|:------------|:-----------------|:--------|
| Silt | Silt Percentage | % |
| Sand | Sand Percentage | % |
| Clay | Clay Percentage | % |
| bd | Bulk Density | g/cm3 |
| theta_s | Saturated Soil Water Content | m3/m3 |
| theta_r | Residual Soil Water Content | m3/m3 |
| ksat | Saturated Hydraulic Conductivity | cm/hr |
| pH | Soil pH in H20 | N/A |
| om | Organic Matter | % |
| lambda | Pore Size Distribution Index | N/A |
| hb | Bubbling Pressure | kPA |
| n | Pore Size Distribution Measure | N/A |
|alpha | Inverse of Mean Pore Diameter | kPa-1 |

Data Citation:
*Chaney, N. W., Wood, E. F., McBratney, A. B., Hempel, J. W., Nauman, T. W., Brungard, C. W., & Odgers, N. P. (2016). POLARIS: A 30-meter probabilistic soil series map of the contiguous United States. Geoderma, 274, 54-67. https://doi.org/10.1016/j.geoderma.2016.03.025*

Psuedocode:

Cycle through each grassland unit...
1. Define soil parameters for data download
2. Determine the simplified geometric min/max latitude and longitude for a given grassland unit
3. Insert the simplified lat/lon into the Polaris URL to determine the specific files to extract
4. Use the data directory defined at the top of code to determine if data
already exists. If not, download the data. Otherwise print a message
5. Load and merge (if necessary) data arrays 
6. Clip arrays to the grassland unit boundary and save


In [ ]:
"""
Generate the POLARIS Soil Data Download URL

Instructions:
Select the desired soil properties for download from the parameters below.

Parameters
==========
soil_var : str
    Soil specific variable (alpha, bd, clay, hb, ksat, lambda, n, om, 
    ph, sand, silt, theta_r, theta_s, vrt)
stat_var : str
    Statistics (mean, mode, p5, p50, p95)
soil_depth : str
    Select a soil depth range (0_5, 5_15, 15_30, 30_60, 60_100, 100_200)

Notes
==========
Minimum/Maximum latitudes and longitude are calculated from the bounds
of the individual grassland unit boundaries. Coordinate indices are 
min [0] and max [1] longitude and min [2] and max [3] latitude.

"""
# Define specific soil parameters for data download
soil_var = 'ph'
stat_var = 'mean'
soil_depth = '60_100'

# Generate URL for each grassland unit and assign to variable
unit_count = 0

while unit_count < len(coord_list):

    # Extract Lat/Lon bound parameters for each Grassland Unit
    lat_min, lon_min = (int(coord_list[unit_count][2]), 
                        int(coord_list[unit_count][0]))
    lat_max, lon_max = lat_min+1, lon_min+1

    # Generate a download URL from the parameters specified above
    polaris_url_format = ('http://hydrology.cee.duke.edu/POLARIS/'
                'PROPERTIES/v1.0/{soil_var}/{stat_var}/{soil_depth}/'
                'lat{lat_min}{lat_max}_lon{lon_min}{lon_max}.tif')
    polaris_url = polaris_url_format.format(
        soil_var=soil_var, stat_var=stat_var, soil_depth=soil_depth,
        lat_min=lat_min, lon_min=lon_min, lat_max=lat_max, lon_max=lon_max)

    if unit_count == 0:
        butte_url = polaris_url
        print('Generating Butte Valley POLARIS URL...')
        print(butte_url)
    elif unit_count == 1:
        crooked_url = polaris_url
        print('Generating Crooked River POLARIS URL...')
        print(crooked_url)
    else:
        print("Problem generating URL. Confirm number of units.")
    
    unit_count = unit_count + 1

In [ ]:
# Download POLARIS Soil Data if it does not exist

for unit_name, row in unit_bndry.iterrows():
    print(f'\nGrassland Unit Name: {unit_name}')
    # Define data directory paths
    soil_path = os.path.join(soil_dir,
                             unit_name.lower().replace(' ', '-'))
    # tif_paths = glob(os.path.join(soil_path, '*.tif'))
    if unit_name == 'Butte Valley National Grassland':
        butte_da = rxr.open_rasterio(
            butte_url, masked=True).squeeze()
        print(butte_da)
    elif unit_name == 'Crooked River National Grassland':
        crooked_da = rxr.open_rasterio(
            crooked_url, masked=True).squeeze()
        print(crooked_da)
    else:
        print ("All data has been downloaded")
    

# rxr.open_rasterio(crooked_url, masked=True).squeeze()

# rxr.open_rasterio(polaris_url, masked=True).squeeze()

In [ ]:
soil_plot = (butte_da.hvplot(
    x='x', y='y',
    xlabel='Longitude (decimal degrees)',
    ylabel='Latitude (decimal degrees)',
    title=('POLARIS Soil Data for Butte Valley National Grassland'
           '\nMean Soil pH for a Soil Depth of 60-100 cm'),
    # aspect='equal',
    # height=600,
    # width=600,
    # xformatter='%.0f',
    # yformatter='%.0f',
    xlim=(-122, -121),
    ylim =(41, 42),
    geo=True,
    cmap='plasma',
    dynamic=False
)
# *
# unit_bndry.loc[['Butte Valley National Grassland']].hvplot(
#     geo=True, fill_color='none')
# )

soil_plot

### SRTM Elevation Data

Elevation data for this model comes from NASA's Shuttle Radar Topography Mission (STRM). The [SRTMGL1 v003](https://lpdaac.usgs.gov/products/srtmgl1v003/) imagery is a 1 arc second (~30 meter) spatial resolution dataset collected as part of a collaborative effort between several foreign and domestic agencies to create a near-global digital elevation model of the Earth. SRTM data are collected in 225 km overlapping swaths at an altitude of 233 km. The coverage area accounts for ~80% of Earth's total landmass and contains data between 60°N and 56°S latitude. STRM version 3 eliminates previously existing void in the dataset.

Data Citation:
*NASA JPL (2013). <i>NASA Shuttle Radar Topography Mission Global 1 arc second</i> [Data set]. NASA EOSDIS Land Processes Distributed Active Archive Center. Accessed 2023-12-10 from https://doi.org/10.5067/MEaSUREs/SRTM/SRTMGL1.003*

In [ ]:
# Documentation for Appeears downloader
# help(eaapp.AppeearsDownloader)

In [ ]:
butte_bndry = (
    unit_bndry
    .loc[['Butte Valley National Grassland']])
# butte_bndry

download_key='butte-valley-srtm'
elev_data_path = os.path.join(elevation_dir, download_key)

if not os.path.exists(elev_data_path):
    print('Downloading SRTM Data...')
    srtm_downloader = eaapp.AppeearsDownloader(
        download_key=download_key,
        ea_dir=elevation_dir,
        product='SRTMGL1_NC.003',
        layer='SRTMGL1_DEM',
        start_date='02-11-2000',
        end_date='02-21-2000',
        polygon=butte_bndry
    )
    srtm_downloader.download_files()
else:
    print('SRTM data has already been downloaded.')

In [ ]:
srtm_paths = glob(
    os.path.join(
        elev_data_path,
        'SRTMGL1_NC.003*',
        '*.tif'))

[rxr.open_rasterio(srtm_path, masked=True).squeeze() for srtm_path in srtm_paths]

### MACAv2 Climate Data

MACA is a statistical method for downscaling Global Climate Models from traditional coarse, lower resolutions into finer, higher resolution derivatives. Variables that are downscaled in the MACA method include daily 2-meter minimum/maximum temperature, relative humidity, and specific humidity, 10-meter zonal and meridonal wind, downward shortwave radiation at the surface, and precipitation accumulation.

The MACAv2 dataset has the following properties:

| Property | Description |
|:---------|:------------|
|Spatial Resolution| 4-km |
|Spatial Projection| WGS84 |
|Spatial Extent| Contiguous USA (CONUS) |
|Temporal Resolution| Daily |
|Model Temporal Extent| 1950-2099 |


MACA Downscaled Variables:

| Variable | Description |
|:---------|:------------|
|tasmax| Maximum daily temperature near surface |
|tasmin| Minimum daily temperature near surface |
|rhsmax| Maximum daily relative humidity near surface |
|rhsmin| Minimum daily relative humdity near surface |
|huss| Average daily specific humidity near surface |
|pr| Average daily precipitation amount at surface |
|rsds| Average daily downward shortwave radiation at surface |
|was| Average daily wind speed near surface |
|uas| Average daily eastward component of wind near surface |
|vas| Average daily northward component of wind near surface |


The dataset used in this analysis comes from the [Northwest Knowledge Network](https://climate.northwestknowledge.net/MACA/data_catalogs.php). The [MACAv2-METDATA](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_aggregated_macav2_monthly_catalog.html) datast used is an aggregated dataset (includes all years 1950-2005 and 2006-2100) covering monthly values. Daily values are available, but are not used here. For climate variables assessed, we look at:

1. REACCH MACAV2 Monthly Aggregation by Precipitation - rcp45 - which represents a precipitation projection assuming a moderate CO2 emissions scenario.

    [MACAV2-METDATA pr MRI-CGCM3 rcp45 2006-2099 Aggregated Monthly](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_aggregated_macav2_monthly_catalog.html?dataset=agg_macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp45_2006_2099_CONUS_monthly)

2. REACCH MACAV2 Monthly Aggregation by precipitation - rcp85 - which represents a precipitation projection assuming a high CO2 emissons scenario.

    [MACAV2-METDATA pr MRI-CGCM3 rcp85 2006-2099 Aggregated Monthly](http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_aggregated_macav2_monthly_catalog.html?dataset=agg_macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2006_2099_CONUS_monthly)


Data Citation:
*Abatzoglou J.T. and Brown T.J. (2012). "A comparison of statistical downscaling methods suited for wildfire applications". International Journal of Climatology, doi: 10.1002/joc.2312*

00:26:00

In [ ]:
# RCP 45 - MRI-CGCM3 Climate Model - Moderate Emissions
# Data used here is from 2006 only

maca_url =('http://thredds.northwestknowledge.net:8080/thredds/ncss/agg_'
           'macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp45_2006_2099_CONUS_monthly.nc'
           '?var=precipitation'
           '&disableLLSubset=on'
           '&disableProjSubset=on'
           '&horizStride=1'
           '&time_start=2006-01-15T00%3A00%3A00Z'
           '&time_end=2006-12-15T00%3A00%3A00Z'
           '&timeStride=1'
           '&accept=netcdf')
# Request access to MACA data through URL
maca_response = requests.get(maca_url)

# Need to save this data in the climate directory
# Open the file for writing with binary (wb)
with open('maca.nc', 'wb') as maca_file:
    maca_file.write(maca_response.content)


In [ ]:
# Save maca dataset to variable
maca_ds = xr.open_dataset('maca.nc')
# Reassign coordinates so lon was in correct range (negative)
maca_ds = maca_ds.assign_coords(lon = maca_ds.lon-360)
# Pull out precipitation variable to work with array directly
precip45 = maca_ds.precipitation
# Write CRS to the precipitation data
precip45.rio.write_crs("epsg:4326", inplace = True)
# Set the spatial dimension of the precipitation data
precip45.rio.set_spatial_dims('lon', 'lat', inplace = True)

# Plot mean precipitation for the given year
precip45.mean('time').hvplot(rasterize=True) * butte_bndry.hvplot()

In [ ]:
precip45.rio.clip_box(*butte_bndry.total_bounds).mean('time').hvplot()

In [ ]:
# RCP 85 - MRI-CGCM3 Climate Model - Moderate Emissions